# PROJET SEATTLE ENERGY BENCHMARKING
## Notebook 01 : Évaluation  de la qualité des données

---

### Identité du document
* **Statut :** Phase 1 (exploration & prototypage)
* **Date de création :** 29 Décembre 2025
* **Dernière mise à jour :** 31 Décembre 2025
* **Dépendances notebooks** : Notebook 0

### Description
Effectuer un audit exhaustif de la qualité des données pour identifier les problèmes potentiels avant toute analyse approfondie. Cela permet d'éviter des biais dans les étapes ultérieures et de documenter un plan de nettoyage clair.

> **Philosophie :** "Mieux vaut détecter les problèmes maintenant que découvrir qu'on a analysé des données biaisées après 2 semaines de travail."

### Objectifs principaux
1. **Cartographier** la complétude des données (analyse des NaN).
2. **Distinguer** les erreurs manifestes des valeurs extrêmes légitimes.
3. **Valider** la cohérence interne (logique métier et physique).
4. **Évaluer** l'impact du flag `DefaultData`.
5. **Produire** un plan d'action pour le nettoyage.

### Dépendances critiques
* `hydra` : Gestion de configuration.
* `pandas`, `seaborn`, `matplotlib` : Manipulation et inspection visuelle.
* `src.data_quality/` : Fonctions à construire.

### LIVRABLES
1. **Rapport de Qualité**  : Synthèse des constats.
2. **Matrice de Stratégie**  : Plan d'action ligne par ligne.
3. **Figures d'audit** : Preuves visuelles.

---

---
# Table des Matières
- [Section 0 : Importation et chargement](#section-0--importation-et-chargement)
- [Section 1 : Où sont les données manquantes et pourquoi ? (Completeness)](#section-1--où-sont-les-données-manquantes-et-pourquoi)
- [Section 2 : Les valeurs extrêmes sont-elles des erreurs ou des réalités ? (Outliers)](#section-2--les-valeurs-extrêmes-sont-elles-des-erreurs-ou-des-réalités)
- [Section 3 : Les données respectent-elles la logique physique et métier ? (Consistency)](#section-3--les-données-respectent-elles-la-logique-physique-et-métier)
- [Section 4 : Le flag "DefaultData" compromet-il la fiabilité ? (Reliability)](#section-4--le-flag-defaultdata-compromet-il-la-fiabilité)
- [Section 5 : Existe-t-il des doublons ou des quasi-doublons ? (Uniqueness)](#section-5--existe-t-il-des-doublons-ou-des-quasi-doublons)
- [Section 6 : Quelle stratégie de nettoyage adopter ? (Action Plan)](#section-6--quelle-stratégie-de-nettoyage-adopter)

---

# Section 0 : Importation et chargement

## Importation des packages

In [1]:
import logging
import pandas as pd
from pathlib import Path


# Import des fonctions utilitaires 

import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
SRC_PATH = PROJECT_ROOT / "src"

if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))


from data.load_data import load_data_raw
from utils.config_loader import load_config, create_directories
from utils.eda_logger import setup_eda_logger
 


## init config et chargement

In [2]:
# Chargement de la configuration principale (Hydra)
cfg = load_config()

# Initialisation du logger
setup_eda_logger(cfg)
logger = logging.getLogger(__name__)

# Création des dossiers nécessaires à l'exécution (raw, interim, processed, reports, etc.)
create_directories(cfg)
# Chargement des données brutes

# Forcer pandas à afficher toutes les colonnes
pd.set_option("display.max_columns", None)

# Forcer pandas à afficher toutes les lignes (si besoin)
pd.set_option("display.max_rows", None)

# Forcer pandas à afficher toute la largeur (évite les "...")
pd.set_option("display.width", None)



df_raw = load_data_raw(cfg)

# Validation dimensions
n_rows, n_cols = df_raw.shape
logger.info(f"Dataset chargé ({n_rows} lignes, {n_cols} colonnes)")

# Premier aperçu
df_raw.sample(5)


2025-12-31 13:56:34,661 - utils.config_loader - INFO - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\data\raw
2025-12-31 13:56:34,663 - utils.config_loader - INFO - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\data\interim
2025-12-31 13:56:34,664 - utils.config_loader - INFO - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\data\processed
2025-12-31 13:56:34,666 - utils.config_loader - INFO - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\figures
2025-12-31 13:56:34,667 - utils.config_loader - INFO - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\reports
2025-12-31 13:56:34,707 - data.load_data - INFO - DataFrame chargé : 3376 lignes, 46 colonnes
2025-12-31 13:56:34,727 - data.load_data - INFO -  Intégrité des données validée (Aucune modification détectée).
2025-12-31 13:56:34,728 - __main__ - INFO - Dataset chargé (

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,CouncilDistrictCode,Neighborhood,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearsENERGYSTARCertified,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
1659,23022,2016,Nonresidential COS,Other,Northgate Campus,10510 5th Ave NE,Seattle,WA,98125.0,2926049431,5,NORTH,47.70541,-122.32232,2005,2.0,1,30498,0,30498,Other - Recreation,Other - Recreation,30498.0,NaN,NaN,NaN,NaN,NaN,NaN,79.900002,87.000000,156.899994,164.399994,2436063.000,2.652254e+06,0.0,312492.9063,1066226.0,13698.37012,1369837.0,False,NaN,Compliant,NaN,80.19,2.63
532,719,2016,NonResidential,Distribution Center,MACMILLAN-PIPER EYRES WAREHOUSE,1762 6th Ave S,Seattle,WA,98134.0,7666203440,2,GREATER DUWAMISH,47.58685,-122.32501,1960,1.0,1,110613,0,110613,"Distribution Center, Office",Distribution Center,99551.0,Office,11061.0,NaN,NaN,NaN,95.0,9.200000,9.200000,28.799999,28.799999,1015060.188,1.015060e+06,0.0,297497.0938,1015060.0,0.00000,0.0,False,NaN,Compliant,NaN,7.08,0.06
1551,22372,2016,Multifamily LR (1-4),Low-Rise Multifamily,1400 TAYLOR CONDOMINIUM,1400 Taylor Ave N,Seattle,WA,98109.0,2617550000,7,MAGNOLIA / QUEEN ANNE,47.63162,-122.34582,1984,1.0,3,23210,0,23210,"Multifamily Housing, Parking",Multifamily Housing,21788.0,Parking,1422.0,NaN,NaN,NaN,69.0,27.600000,29.299999,86.699997,92.000000,601818.625,6.385772e+05,0.0,176382.9063,601818.0,0.00000,0.0,False,NaN,Compliant,NaN,4.20,0.18
637,866,2016,SPS-District K-12,K-12 School,Lafayette Elementary,2645 California Ave. S.W.,Seattle,WA,98116.0,9277200005,1,SOUTHWEST,47.57950,-122.38723,1950,1.0,1,61183,0,61183,K-12 School,K-12 School,62975.0,NaN,NaN,NaN,NaN,NaN,92.0,38.500000,44.299999,64.300003,71.599998,2424724.250,2.789344e+06,0.0,211139.0000,720406.0,17043.17773,1704318.0,True,NaN,Error - Correct Default Data,NaN,95.54,1.56
1412,21731,2016,Multifamily MR (5-9),Mid-Rise Multifamily,The Humphrey,2205 2nd Ave,Seattle,WA,98121.0,2467400181,7,LAKE UNION,47.62444,-122.33261,2001,1.0,6,79402,0,79402,"Multifamily Housing, Retail Store",Multifamily Housing,49049.0,Retail Store,1327.0,NaN,NaN,NaN,NaN,34.700001,37.500000,108.900002,117.699997,1747487.750,1.888622e+06,0.0,512159.3125,1747488.0,0.00000,0.0,False,NaN,Compliant,NaN,12.18,0.15


# Section 1 : Cartographie des valeurs manquantes

**Objectif :** Quantifier et visualiser les valeurs manquantes pour identifier les patterns et mécanismes sous-jacents. Cela aide à évaluer la complétude du dataset et à anticiper les biais.

## Calcul et tri des pourcentages de NaN

Nous calculons le % de NaN par colonne et trions par ordre décroissant.
Colonnes avec >50% de NaN sont candidates à suppression.

In [5]:
# df_raw['Comments'].isnull()

In [12]:
# Calcul des NaN
missing_perc = df_raw.isnull().mean() * 100 # mean() calcule la proportion de NaN par colonne
missing_df = pd.DataFrame({'Column': missing_perc.index, '% Missing': missing_perc.values})
missing_df = missing_df.sort_values(by='% Missing', ascending=False).reset_index(drop=True)

logger.info("Colonnes avec le plus de NaN :")
display(missing_df.head(30))

# Identification colonnes >50% NaN
high_missing = missing_df[missing_df['% Missing'] > 50]
logger.info(f"Colonnes avec >50% NaN : {high_missing['Column'].tolist()}")

2025-12-31 13:39:33,582 - __main__ - INFO - Colonnes avec le plus de NaN :


,Column,% Missing
0,Comments,100.000000
1,Outlier,99.052133
2,YearsENERGYSTARCertified,96.475118
3,ThirdLargestPropertyUseType,82.345972
4,ThirdLargestPropertyUseTypeGFA,82.345972
5,SecondLargestPropertyUseType,50.266588
6,SecondLargestPropertyUseTypeGFA,50.266588
7,ENERGYSTARScore,24.970379
8,LargestPropertyUseTypeGFA,0.592417
9,LargestPropertyUseType,0.592417


2025-12-31 13:39:33,591 - __main__ - INFO - Colonnes avec >50% NaN : ['Comments', 'Outlier', 'YearsENERGYSTARCertified', 'ThirdLargestPropertyUseType', 'ThirdLargestPropertyUseTypeGFA', 'SecondLargestPropertyUseType', 'SecondLargestPropertyUseTypeGFA']


> techniquement ([pandas.isnull()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.isnull.html)) permet de detecter la plus part des valeurs manquantes,si on en oubli certains ,great expectation pourra les détecter

On constate que beaucoup de variables ont des valeurs manquantes, notamment 7 colonnes critiques qui dépassent les 50 % de NAN. Les variables `Comments`, `Outlier` et `YearsENERGYSTARCertified` sont quasiment inexistantes (entre 96 % et 100 % de manque), ce qui les rend statistiquement inexploitables.

De même, les données liées aux usages secondaires et tertiaires des bâtiments présentent des taux de vacance massifs, révélant que la majorité des structures déclarées n'ont qu'une seule activité principale ou que ces informations n'ont pas été jugées nécessaires lors de la saisie.

Le **ENERGYSTARScore** constitue un point de vigilance majeur avec environ 25 % de données manquantes, un constat qui suggère qu'un quart du parc immobilier de Seattle n'est pas éligible à cette notation ou possède des relevés incomplets.

À l'inverse, on peut noter que les variables comme la consommation réelle d'énergie et les émissions de gaz à effet de serre sont assez robustes, avec moins de 1 % de NA. Cette solidité sur les indicateurs de performance réels garantit que, malgré l'élimination nécessaire des colonnes lacunaires, le cœur du dataset restera parfaitement fiable en vue de la modélisation.

In [5]:
# code

## Section 2 : Les valeurs extrêmes sont-elles des erreurs ou des réalités ?